# Основные разделы работы с текстом

Рассмотрим подходы и доступные библиотеки, позволяющие собирать признаки из текстов

In [1]:
!pip install pymorphy3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.3 MB/s eta 0:00:00


In [2]:
!pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=cd435a3bd5380f6597f033bc143282fb2c3ea212b473e311e84a711ccb0080c5
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=1119925db03a855688df354715a03da76cdce1f30bfdcdcfaeb7a9ad4728620e
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [170]:
import re # регулярные выражения: предобработка текстов, строить признаки-шаблоны на слова и подстроки текста и многое другое
import string # работа со строками (в тч пунктуация)

import nltk # пакет для символьной и статистической обработки естественного языка
from nltk.tokenize import word_tokenize, sent_tokenize #токенизация текста (слов, предложений)
from nltk.corpus import stopwords # набор стоп-слов из библиотеки

from nltk.tokenize.casual import TweetTokenizer # токенизатор с поддержкой особенностей коротких сообщений Twitter
from nltk.tokenize import MWETokenizer # берет строку, которая уже была разделена на токены,
# и повторно токенизирует ее, объединяя выражения из нескольких слов в отдельные токены, используя словарь MWE

from nltk.stem import PorterStemmer # Метод, который принимает слово в качестве входного аргумента и возвращает слово, уменьшается до его корневой формы
from nltk.stem import WordNetLemmatizer # проставить соответствующий POS-тег для каждого слова при обработке текстов

from nltk.util import ngrams # модель n-грамм (последовательность из n элементов, символов строки/слова)

# brown корпус, входящим в состав NLTK.
# Он размечен тегами частей речи
from nltk.corpus import brown

# Корпус UnigramTagger - теггер, который выбирает для каждого слова ту часть речи,
# с которой оно чаще всего встречалось в обучающем корпусе.
from nltk.tag import UnigramTagger

import pymorphy3 # Для русского и украинского языка

# объект для лемматизации MorphVocab
# извлечение эмбеддингов из текста NewsEmbedding
# объект для синтаксического анализа NewsSyntaxParser
# извлечение имен собственных из текста NamesExtractor
# извлечение тэгов по морфологии NewsMorphTagger
# Segmenter
# создание объекта документа (можно создать из предложения, потом с ним работать и парсить, анализировать) Doc
from natasha import MorphVocab, NewsEmbedding, NewsSyntaxParser, NamesExtractor, NewsMorphTagger, Doc, Segmenter, NewsNERTagger

from collections import Counter # превращение в словарь

In [4]:
morph = pymorphy3.MorphAnalyzer() # Создать экземпляр Морфологического анализатора для русского языка

In [5]:
# загрузим модули nltk
#Удаление знаков препинания
nltk.download('punkt')

# Удаление стоп-слов
nltk.download('stopwords')

# Лексическая БД английского языка
nltk.download('wordnet')

# Разметчик использует алгоритм персептрона, чтобы определить, какой POS-тег присвоить тому или иному слову
nltk.download('averaged_perceptron_tagger')

# Загрузить список доступных тэгов для POS обработки
nltk.download('tagsets')

# Скачать корпус brown
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

## NLTK

Распространенная библиотека для обработки текстов. Содержит алгоритмы, корпусы, обученные модели.
http://www.nltk.org
Есть книга по обработке текста с использованием NLTK: http://www.nltk.org/book/

Вместе с nltk идет набор данных, который не скачивается по-умолчанию.
Для выполнения кода, приведенного ниже может понадобиться скачать дополнительные файлы. Для этого запустите код:

In [6]:
nltk.download() # Откроется обычное GUI-окно. Закройте его, чтобы продолжить выполнение ноутбука.

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [7]:
song_data = ""
with open("/content/song_data.txt", encoding='utf-8') as f:
  song_data=f.read()

In [8]:
song_data

'Чёрный ворон, чёрный ворон,\nЧто ты вьёшься надо мной?\nТы добычи не дождёшься,\nЧёрный ворон, я не твой!\n\nЧто ты когти распускаешь\nНад моею головой?\nИль добычу себе чаешь,\nЧёрный ворон, я не твой!\n\nЗавяжу смертельну рану\nПодарённым мне платком,\nА потом с тобой я стану\nГоворить всё об одном.\n\nПолети в мою сторонку,\nСкажи маменьке моей,\nТы скажи моей любезной,\nЧто за Родину я пал.\n\nОтнеси платок кровавый\nМилой любушке моей.\nТы скажи – она свободна,\nЯ женился на другой.\n\nВзял невесту тиху-скромну\nВ чистом поле под кустом,\nОбвенчальна была сваха –\nСабля вострая моя.\n\nКалена стрела венчала\nСреди битвы роковой.\nВижу смерть моя приходит –\nЧёрный ворон, весь я твой!\n\nКак-то летом на рассвете\nЗаглянул в соседний сад,\nТам смуглянка-молдаванка\nСобирает виноград.\nЯ краснею, я бледнею,\nЗахотелось вдруг сказать:\n«Станем над рекою\nЗорьки летние встречать!»\n\nПрипев:\n\nРаскудрявый клён зелёный, лист резной,\nЯ влюблённый и смущённый пред тобой.\nКлён зелёный,

In [17]:
song_data = song_data.replace("\n"," ")

In [11]:
song_data_sent = ""
with open("/content/song_data.txt", encoding='utf-8') as f:
  song_data_sent=f.read()

song_data_sent = song_data_sent.replace("\n"," ")
song_data_sent

'Чёрный ворон, чёрный ворон, Что ты вьёшься надо мной? Ты добычи не дождёшься, Чёрный ворон, я не твой!  Что ты когти распускаешь Над моею головой? Иль добычу себе чаешь, Чёрный ворон, я не твой!  Завяжу смертельну рану Подарённым мне платком, А потом с тобой я стану Говорить всё об одном.  Полети в мою сторонку, Скажи маменьке моей, Ты скажи моей любезной, Что за Родину я пал.  Отнеси платок кровавый Милой любушке моей. Ты скажи – она свободна, Я женился на другой.  Взял невесту тиху-скромну В чистом поле под кустом, Обвенчальна была сваха – Сабля вострая моя.  Калена стрела венчала Среди битвы роковой. Вижу смерть моя приходит – Чёрный ворон, весь я твой!  Как-то летом на рассвете Заглянул в соседний сад, Там смуглянка-молдаванка Собирает виноград. Я краснею, я бледнею, Захотелось вдруг сказать: «Станем над рекою Зорьки летние встречать!»  Припев:  Раскудрявый клён зелёный, лист резной, Я влюблённый и смущённый пред тобой. Клён зелёный, да клён кудрявый, Да раскудрявый, резной. Раску

## Токенизация

Разбиение текста на токены (слова, пунктуация, числа, url, телефоны, ...)

http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize

In [12]:
text = "Natural Language Toolkit (NLTK) is a Python package for natural language processing. It requires Python 3.7, 3.8, 3.9, 3.10 or 3.11"

print(sent_tokenize(text))
print(word_tokenize(text))

['Natural Language Toolkit (NLTK) is a Python package for natural language processing.', 'It requires Python 3.7, 3.8, 3.9, 3.10 or 3.11']
['Natural', 'Language', 'Toolkit', '(', 'NLTK', ')', 'is', 'a', 'Python', 'package', 'for', 'natural', 'language', 'processing', '.', 'It', 'requires', 'Python', '3.7', ',', '3.8', ',', '3.9', ',', '3.10', 'or', '3.11']


Нужно следить, чтобы токенизация правильно работала на ваших текстах.
Например, на текстах из twitter стандартный токенизатор может работать плохо:

In [13]:
text = "hello @username!!!!! :) http://alink.com/page#part #hashtag"
print(word_tokenize(text))

['hello', '@', 'username', '!', '!', '!', '!', '!', ':', ')', 'http', ':', '//alink.com/page', '#', 'part', '#', 'hashtag']


In [14]:
tokenizer = TweetTokenizer(reduce_len=True)
tokenizer.tokenize(text)

['hello',
 '@username',
 '!',
 '!',
 '!',
 ':)',
 'http://alink.com/page#part',
 '#hashtag']

Иногда помогает объединить несколько токенов в один. Например, устойчивые выражения или конструкции вида $2000, 40x50.

In [15]:
tokenizer = MWETokenizer([('a', 'little'), ('a', 'little', 'bit'), ('a', 'lot')])
tokenizer.add_mwe(('in', 'spite', 'of'))
tokenizer.tokenize(word_tokenize('In a little or a little bit or a lot in spite of'))

['In', 'a_little', 'or', 'a_little_bit', 'or', 'a_lot', 'in_spite_of']

In [18]:
song_data

'Чёрный ворон, чёрный ворон, Что ты вьёшься надо мной? Ты добычи не дождёшься, Чёрный ворон, я не твой!  Что ты когти распускаешь Над моею головой? Иль добычу себе чаешь, Чёрный ворон, я не твой!  Завяжу смертельну рану Подарённым мне платком, А потом с тобой я стану Говорить всё об одном.  Полети в мою сторонку, Скажи маменьке моей, Ты скажи моей любезной, Что за Родину я пал.  Отнеси платок кровавый Милой любушке моей. Ты скажи – она свободна, Я женился на другой.  Взял невесту тиху-скромну В чистом поле под кустом, Обвенчальна была сваха – Сабля вострая моя.  Калена стрела венчала Среди битвы роковой. Вижу смерть моя приходит – Чёрный ворон, весь я твой!  Как-то летом на рассвете Заглянул в соседний сад, Там смуглянка-молдаванка Собирает виноград. Я краснею, я бледнею, Захотелось вдруг сказать: «Станем над рекою Зорьки летние встречать!»  Припев:  Раскудрявый клён зелёный, лист резной, Я влюблённый и смущённый пред тобой. Клён зелёный, да клён кудрявый, Да раскудрявый, резной. Раску

In [19]:
print(sent_tokenize(song_data))
print(word_tokenize(song_data))

['Чёрный ворон, чёрный ворон, Что ты вьёшься надо мной?', 'Ты добычи не дождёшься, Чёрный ворон, я не твой!', 'Что ты когти распускаешь Над моею головой?', 'Иль добычу себе чаешь, Чёрный ворон, я не твой!', 'Завяжу смертельну рану Подарённым мне платком, А потом с тобой я стану Говорить всё об одном.', 'Полети в мою сторонку, Скажи маменьке моей, Ты скажи моей любезной, Что за Родину я пал.', 'Отнеси платок кровавый Милой любушке моей.', 'Ты скажи – она свободна, Я женился на другой.', 'Взял невесту тиху-скромну В чистом поле под кустом, Обвенчальна была сваха – Сабля вострая моя.', 'Калена стрела венчала Среди битвы роковой.', 'Вижу смерть моя приходит – Чёрный ворон, весь я твой!', 'Как-то летом на рассвете Заглянул в соседний сад, Там смуглянка-молдаванка Собирает виноград.', 'Я краснею, я бледнею, Захотелось вдруг сказать: «Станем над рекою Зорьки летние встречать!»  Припев:  Раскудрявый клён зелёный, лист резной, Я влюблённый и смущённый пред тобой.', 'Клён зелёный, да клён кудряв

Задача сложна. Ещё больше задачу усложняют - опечатки. В некоторых случаях правильное деление на токены можно выполнить только с учетом знаний о морфологии, синтаксисе.

## Стоп-слова

Списки стоп-слов используются для того, чтобы игнорировать слова, которые не помогают в решении задачи.

Это могут быть конструкции, которые не несут самостоятельного смысла в предложении. Для классификации и кластеризации могут быть бесполезны высокочастотные слова, которые есть почти в каждом тексте.

Фильтрация стоп-слов иногда помогает повысить качество

In [20]:
russian_stopwords = stopwords.words("russian") #задать язык
for w in russian_stopwords:
    print(w, end=" ")


и в во не что он на я с со как а то все она так его но да ты к у же вы за бы по только ее мне было вот от меня еще нет о из ему теперь когда даже ну вдруг ли если уже или ни быть был него до вас нибудь опять уж вам ведь там потом себя ничего ей может они тут где есть надо ней для мы тебя их чем была сам чтоб без будто чего раз тоже себе под будет ж тогда кто этот того потому этого какой совсем ним здесь этом один почти мой тем чтобы нее сейчас были куда зачем всех никогда можно при наконец два об другой хоть после над больше тот через эти нас про всего них какая много разве три эту моя впрочем хорошо свою этой перед иногда лучше чуть том нельзя такой им более всегда конечно всю между 

In [21]:
len(russian_stopwords)

151

In [ ]:
russian_stopwords

In [23]:
russian_stopwords.extend(['это', 'нею'])

In [73]:
russian_stopwords.extend(['иль', 'всё'])

In [74]:
len(russian_stopwords)

155

In [67]:
tree_list = song_data.split()
tree_list

['Чёрный',
 'ворон,',
 'чёрный',
 'ворон,',
 'Что',
 'ты',
 'вьёшься',
 'надо',
 'мной?',
 'Ты',
 'добычи',
 'не',
 'дождёшься,',
 'Чёрный',
 'ворон,',
 'я',
 'не',
 'твой!',
 'Что',
 'ты',
 'когти',
 'распускаешь',
 'Над',
 'моею',
 'головой?',
 'Иль',
 'добычу',
 'себе',
 'чаешь,',
 'Чёрный',
 'ворон,',
 'я',
 'не',
 'твой!',
 'Завяжу',
 'смертельну',
 'рану',
 'Подарённым',
 'мне',
 'платком,',
 'А',
 'потом',
 'с',
 'тобой',
 'я',
 'стану',
 'Говорить',
 'всё',
 'об',
 'одном.',
 'Полети',
 'в',
 'мою',
 'сторонку,',
 'Скажи',
 'маменьке',
 'моей,',
 'Ты',
 'скажи',
 'моей',
 'любезной,',
 'Что',
 'за',
 'Родину',
 'я',
 'пал.',
 'Отнеси',
 'платок',
 'кровавый',
 'Милой',
 'любушке',
 'моей.',
 'Ты',
 'скажи',
 '–',
 'она',
 'свободна,',
 'Я',
 'женился',
 'на',
 'другой.',
 'Взял',
 'невесту',
 'тиху-скромну',
 'В',
 'чистом',
 'поле',
 'под',
 'кустом,',
 'Обвенчальна',
 'была',
 'сваха',
 '–',
 'Сабля',
 'вострая',
 'моя.',
 'Калена',
 'стрела',
 'венчала',
 'Среди',
 'битвы',


In [68]:
song_data_tokens = word_tokenize(song_data)

In [75]:
cleaned_song_data = [word if word.lower() not in russian_stopwords else "" for word in song_data_tokens]
cleaned_song_data

['Чёрный',
 'ворон',
 ',',
 'чёрный',
 'ворон',
 ',',
 '',
 '',
 'вьёшься',
 '',
 'мной',
 '?',
 '',
 'добычи',
 '',
 'дождёшься',
 ',',
 'Чёрный',
 'ворон',
 ',',
 '',
 '',
 'твой',
 '!',
 '',
 '',
 'когти',
 'распускаешь',
 '',
 'моею',
 'головой',
 '?',
 '',
 'добычу',
 '',
 'чаешь',
 ',',
 'Чёрный',
 'ворон',
 ',',
 '',
 '',
 'твой',
 '!',
 'Завяжу',
 'смертельну',
 'рану',
 'Подарённым',
 '',
 'платком',
 ',',
 '',
 '',
 '',
 'тобой',
 '',
 'стану',
 'Говорить',
 '',
 '',
 'одном',
 '.',
 'Полети',
 '',
 'мою',
 'сторонку',
 ',',
 'Скажи',
 'маменьке',
 'моей',
 ',',
 '',
 'скажи',
 'моей',
 'любезной',
 ',',
 '',
 '',
 'Родину',
 '',
 'пал',
 '.',
 'Отнеси',
 'платок',
 'кровавый',
 'Милой',
 'любушке',
 'моей',
 '.',
 '',
 'скажи',
 '–',
 '',
 'свободна',
 ',',
 '',
 'женился',
 '',
 '',
 '.',
 'Взял',
 'невесту',
 'тиху-скромну',
 '',
 'чистом',
 'поле',
 '',
 'кустом',
 ',',
 'Обвенчальна',
 '',
 'сваха',
 '–',
 'Сабля',
 'вострая',
 '',
 '.',
 'Калена',
 'стрела',
 'венчала',

In [76]:
cleaned_song_data_text = " ".join(cleaned_song_data)

cleaned_song_data_text

'Чёрный ворон , чёрный ворон ,   вьёшься  мной ?  добычи  дождёшься , Чёрный ворон ,   твой !   когти распускаешь  моею головой ?  добычу  чаешь , Чёрный ворон ,   твой ! Завяжу смертельну рану Подарённым  платком ,    тобой  стану Говорить   одном . Полети  мою сторонку , Скажи маменьке моей ,  скажи моей любезной ,   Родину  пал . Отнеси платок кровавый Милой любушке моей .  скажи –  свободна ,  женился   . Взял невесту тиху-скромну  чистом поле  кустом , Обвенчальна  сваха – Сабля вострая  . Калена стрела венчала Среди битвы роковой . Вижу смерть  приходит – Чёрный ворон , весь  твой ! Как-то летом  рассвете Заглянул  соседний сад ,  смуглянка-молдаванка Собирает виноград .  краснею ,  бледнею , Захотелось  сказать : « Станем  рекою Зорьки летние встречать ! » Припев : Раскудрявый клён зелёный , лист резной ,  влюблённый  смущённый пред тобой . Клён зелёный ,  клён кудрявый ,  раскудрявый , резной . Раскудрявый клён зелёный , лист резной ,  влюблённый  смущённый пред тобой . Клён зе

## Регулярные выражения

С помощью регулярных выражений можно осуществлять предобработку текстов, строить признаки-шаблоны на слова и подстроки текста и многое другое.
Документация для python:
https://docs.python.org/3.6/howto/regex.html
https://docs.python.org/3.6/library/re.html#module-re

re.X позволяет использовать пробельные символы и делать комментарии в регулярных выражениях

In [169]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

## 1.2. Разбейте данные на предложения

Для токенизации используйте nltk.sent_tokenize. чтобы удалить имен пользователей из сообщений напишите регулярное выражение.

<b>Классы символов в регулярных выражениях</b>:

[A-Z] – символы верхнего регистра (латиница)

[a-z] – символы нижнего регистра (латиница)

[А-Я] – символы верхнего регистра (кириллица)

[а-я] – символы нижнего регистра (кириллица)

[0-9] или \d – цифра

[^0-9] или \D – любой символ, кроме цифры

. – Один любой символ, кроме новой строки \n.

? – 0 или 1 вхождение шаблона слева

\+ – 1 и более вхождений шаблона слева

\* – 0 и более вхождений шаблона слева

\w – Любая цифра или буква (\W — все, кроме буквы или цифры)

\d – Любая цифра [0-9] (\D — все, кроме цифры)

\s – Любой пробельный символ (\S — любой непробельнй символ)

\b – Граница слова

[..] – дин из символов в скобках ([^..] — любой символ, кроме тех, что в скобках)

\ – Экранирование специальных символов (\. означает точку или \+ — знак «плюс»)

^ и $ – Начало и конец строки соответственно

{n,m} – От n до m вхождений ({,m} — от 0 до m)

a|b – Соответствует a или b

() – Группирует выражение и возвращает найденный текст

\t, \n, \r – Символ табуляции, новой строки и возврата каретки соответственно

In [31]:
a = re.compile(r"""\d +  # общая часть
                   \.    # точка
                   \d *  # дробное значение""", re.X)
b = re.compile(r"\d+\.\d*")
a.search("asdf 1.1232")

<re.Match object; span=(5, 11), match='1.1232'>

In [32]:
string.punctuation
'!Hello.'.strip(string.punctuation)
s_test = "Hello, world. I'm a boy, you're a girl."
' '.join(word.strip(string.punctuation) for word in s_test.split())

"Hello world I'm a boy you're a girl"

In [35]:
def remove_punct(text):
    new_words = []
    for word in text:
        w = re.sub(r'[^\w\s]','',word) #оставить только слова и пробелы
        w = re.sub(r'_','',w) #нижнее подчеркивание убираем
        new_words.append(w)
    return "".join(new_words)

In [36]:
remove_punct(s_test)

'Hello world Im a boy youre a girl'

In [77]:
cleaned_song_data_text = cleaned_song_data_text.replace("-", " ")
cleaned_song_data_text

'Чёрный ворон , чёрный ворон ,   вьёшься  мной ?  добычи  дождёшься , Чёрный ворон ,   твой !   когти распускаешь  моею головой ?  добычу  чаешь , Чёрный ворон ,   твой ! Завяжу смертельну рану Подарённым  платком ,    тобой  стану Говорить   одном . Полети  мою сторонку , Скажи маменьке моей ,  скажи моей любезной ,   Родину  пал . Отнеси платок кровавый Милой любушке моей .  скажи –  свободна ,  женился   . Взял невесту тиху скромну  чистом поле  кустом , Обвенчальна  сваха – Сабля вострая  . Калена стрела венчала Среди битвы роковой . Вижу смерть  приходит – Чёрный ворон , весь  твой ! Как то летом  рассвете Заглянул  соседний сад ,  смуглянка молдаванка Собирает виноград .  краснею ,  бледнею , Захотелось  сказать : « Станем  рекою Зорьки летние встречать ! » Припев : Раскудрявый клён зелёный , лист резной ,  влюблённый  смущённый пред тобой . Клён зелёный ,  клён кудрявый ,  раскудрявый , резной . Раскудрявый клён зелёный , лист резной ,  влюблённый  смущённый пред тобой . Клён зе

In [78]:
song_data_wthot_punkt = remove_punct(cleaned_song_data_text).lower() # убрать пунктуацию и привести к нижнему регистру lower

In [79]:
song_data_wthot_punkt

'чёрный ворон  чёрный ворон    вьёшься  мной   добычи  дождёшься  чёрный ворон    твой    когти распускаешь  моею головой   добычу  чаешь  чёрный ворон    твой  завяжу смертельну рану подарённым  платком     тобой  стану говорить   одном  полети  мою сторонку  скажи маменьке моей   скажи моей любезной    родину  пал  отнеси платок кровавый милой любушке моей   скажи   свободна   женился    взял невесту тиху скромну  чистом поле  кустом  обвенчальна  сваха  сабля вострая   калена стрела венчала среди битвы роковой  вижу смерть  приходит  чёрный ворон  весь  твой  как то летом  рассвете заглянул  соседний сад   смуглянка молдаванка собирает виноград   краснею   бледнею  захотелось  сказать   станем  рекою зорьки летние встречать   припев  раскудрявый клён зелёный  лист резной   влюблённый  смущённый пред тобой  клён зелёный   клён кудрявый   раскудрявый  резной  раскудрявый клён зелёный  лист резной   влюблённый  смущённый пред тобой  клён зелёный   клён кудрявый   раскудрявый  резной   

## Стэмминг

— это процесс нахождения основы слова для заданного исходного слова

In [59]:
ps = PorterStemmer()
print(ps.stem("ушла")) # обратить внимание на русский язык
print(ps.stem("детишкам")) # обратить внимание на русский язык
print(ps.stem("keys"))
print(ps.stem("meetings"))

ушла
детишкам
key
meet


##Лемматизация

— процесс приведения словоформы к лемме — её нормальной (словарной) форме

In [61]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("скакал", pos='v'))
print(lemmatizer.lemmatize("ушла", pos='v'))
print(lemmatizer.lemmatize("cooked", pos='v'))
print(lemmatizer.lemmatize("meetings"))
print(lemmatizer.lemmatize("better"))
print(lemmatizer.lemmatize("better", pos="a"))

скакал
ушла
cook
meeting
better
good


## POS-тэггинг

Part of Speech тэггинг позволяет разметить токены тегами частей речи

In [62]:
text = nltk.word_tokenize("She saw the man with a telescope") # токенизация слов
text

['She', 'saw', 'the', 'man', 'with', 'a', 'telescope']

In [81]:
tokenize_tree = [word if word not in russian_stopwords else "" for word in tokenize_tree]
tokenize_tree = " ".join(tokenize_tree)
tokenize_tree

'чёрный ворон чёрный ворон вьёшься мной добычи дождёшься чёрный ворон твой когти распускаешь моею головой добычу чаешь чёрный ворон твой завяжу смертельну рану подарённым платком тобой стану говорить одном полети мою сторонку скажи маменьке моей скажи моей любезной родину пал отнеси платок кровавый милой любушке моей скажи свободна женился взял невесту тиху скромну чистом поле кустом обвенчальна сваха сабля вострая калена стрела венчала среди битвы роковой вижу смерть приходит чёрный ворон весь твой   летом рассвете заглянул соседний сад смуглянка молдаванка собирает виноград краснею бледнею захотелось сказать станем рекою зорьки летние встречать припев раскудрявый клён зелёный лист резной влюблённый смущённый пред тобой клён зелёный клён кудрявый раскудрявый резной раскудрявый клён зелёный лист резной влюблённый смущённый пред тобой клён зелёный клён кудрявый раскудрявый резной смуглянка молдаванка отвечала парню лад партизанский молдаванский собирается отряд нынче рано партизаны дом 

In [84]:
tokenize_tree = nltk.word_tokenize(song_data_wthot_punkt)

In [82]:
nltk.pos_tag(text) # тэггинг

[('She', 'PRP'),
 ('saw', 'VBD'),
 ('the', 'DT'),
 ('man', 'NN'),
 ('with', 'IN'),
 ('a', 'DT'),
 ('telescope', 'NN')]

In [85]:
nltk.pos_tag(tokenize_tree) # тэггинг, NNP: существительное, собственное, единственное число
# прилагательное или числительное, порядковый номер, третий

[('чёрный', 'JJ'),
 ('ворон', 'NNP'),
 ('чёрный', 'NNP'),
 ('ворон', 'NNP'),
 ('вьёшься', 'NNP'),
 ('мной', 'NNP'),
 ('добычи', 'NNP'),
 ('дождёшься', 'NNP'),
 ('чёрный', 'NNP'),
 ('ворон', 'NNP'),
 ('твой', 'NNP'),
 ('когти', 'NNP'),
 ('распускаешь', 'NNP'),
 ('моею', 'NNP'),
 ('головой', 'NNP'),
 ('добычу', 'NNP'),
 ('чаешь', 'NNP'),
 ('чёрный', 'NNP'),
 ('ворон', 'NNP'),
 ('твой', 'NNP'),
 ('завяжу', 'NNP'),
 ('смертельну', 'NNP'),
 ('рану', 'NNP'),
 ('подарённым', 'NNP'),
 ('платком', 'NNP'),
 ('тобой', 'NNP'),
 ('стану', 'NNP'),
 ('говорить', 'NNP'),
 ('одном', 'NNP'),
 ('полети', 'NNP'),
 ('мою', 'NNP'),
 ('сторонку', 'NNP'),
 ('скажи', 'NNP'),
 ('маменьке', 'NNP'),
 ('моей', 'NNP'),
 ('скажи', 'NNP'),
 ('моей', 'NNP'),
 ('любезной', 'NNP'),
 ('родину', 'NNP'),
 ('пал', 'NNP'),
 ('отнеси', 'NNP'),
 ('платок', 'NNP'),
 ('кровавый', 'NNP'),
 ('милой', 'NNP'),
 ('любушке', 'NNP'),
 ('моей', 'NNP'),
 ('скажи', 'NNP'),
 ('свободна', 'NNP'),
 ('женился', 'NNP'),
 ('взял', 'NNP'),
 ('не

In [ ]:
text_rus = nltk.word_tokenize("Она увидела в телескопе человека") # токенизация слов
text_rus

['Она', 'увидела', 'в', 'телескопе', 'человека']

In [ ]:
nltk.pos_tag(text_rus) # тэггинг

[('Она', 'JJ'),
 ('увидела', 'NNP'),
 ('в', 'NNP'),
 ('телескопе', 'NNP'),
 ('человека', 'NN')]

In [ ]:
senten_text = 'В департаменте… но лучше не называть, в каком департаменте. Ничего нет сердитее всякого рода департаментов, полков, канцелярий и, словом, всякого рода должностных сословий. Теперь уже всякий частный человек считает в лице своем оскорбленным все общество.'
tokenize = sent_tokenize(senten_text)  # токенизация предложений
print(tokenize)

['В департаменте… но лучше не называть, в каком департаменте.', 'Ничего нет сердитее всякого рода департаментов, полков, канцелярий и, словом, всякого рода должностных сословий.', 'Теперь уже всякий частный человек считает в лице своем оскорбленным все общество.']


In [ ]:
nltk.pos_tag(tokenize) # тэггинг, неправильный.

[('В департаменте… но лучше не называть, в каком департаменте.', 'JJ'),
 ('Ничего нет сердитее всякого рода департаментов, полков, канцелярий и, словом, всякого рода должностных сословий.',
  'NNP'),
 ('Теперь уже всякий частный человек считает в лице своем оскорбленным все общество.',
  'NN')]

In [86]:
# Получить список доступных тэгов можно с помощью команды:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [87]:
print(nltk.pos_tag(nltk.word_tokenize("A fly to London")))
print(nltk.pos_tag(nltk.word_tokenize("I fly to London")))
print(nltk.pos_tag(nltk.word_tokenize("I opened a window")))

[('A', 'DT'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NNP')]
[('I', 'PRP'), ('fly', 'VBP'), ('to', 'TO'), ('London', 'NNP')]
[('I', 'PRP'), ('opened', 'VBD'), ('a', 'DT'), ('window', 'NN')]


In [88]:
print(nltk.pos_tag(nltk.word_tokenize("I blablablowed the window")))

[('I', 'PRP'), ('blablablowed', 'VBD'), ('the', 'DT'), ('window', 'NN')]


Посмотрим, что если назначать часть речи каждому слову наивно - наиболее частотную часть речи

In [ ]:
# Воспользуемся brown корпусом, входящим в состав NLTK.
# Он размечен тегами частей речи. Посмотрим на несколько первых токенов:
print(brown.tagged_words(categories='news')[:20])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS')]


In [ ]:
# Теперь обучим на корпусе UnigramTagger - теггер, который выбирает для каждого слова ту часть речи,
# с которой оно чаще всего встречалось в обучающем корпусе.
tagger = UnigramTagger(brown.tagged_sents(categories='news')[:3000]) # Возьмем первые 3000 токенов
print(tagger.tag(nltk.word_tokenize("I fly to London")))
print(tagger.tag(nltk.word_tokenize("A fly to London")))

[('I', 'PPSS'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NP')]
[('A', 'AT'), ('fly', 'NN'), ('to', 'TO'), ('London', 'NP')]


In [ ]:
# Проверим качество на отложенном фрагменте brown-корпуса.
tagger.evaluate(brown.tagged_sents(categories='news')[3000:])

<ipython-input-42-614888195e32>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tagger.evaluate(brown.tagged_sents(categories='news')[3000:])


0.7878104465976167

Кажется, что цифра 0.78 достаточно высокая для такого наивного подхода.
Но если задуматься, то это значит, что часть речи каждого пятого слова была определена.

## N-граммы

Для разделения текста на биграммы или триграммы необходимо из пакета nltk загрузить ngrams


In [89]:
text = 'В департаменте… но лучше не называть, в каком департаменте.'
grams = ngrams(text.split(), 2) # биграмма

In [90]:
for gram in grams:
    print(gram)

('В', 'департаменте…')
('департаменте…', 'но')
('но', 'лучше')
('лучше', 'не')
('не', 'называть,')
('называть,', 'в')
('в', 'каком')
('каком', 'департаменте.')


In [91]:
song_data_wthot_punkt

'чёрный ворон  чёрный ворон    вьёшься  мной   добычи  дождёшься  чёрный ворон    твой    когти распускаешь  моею головой   добычу  чаешь  чёрный ворон    твой  завяжу смертельну рану подарённым  платком     тобой  стану говорить   одном  полети  мою сторонку  скажи маменьке моей   скажи моей любезной    родину  пал  отнеси платок кровавый милой любушке моей   скажи   свободна   женился    взял невесту тиху скромну  чистом поле  кустом  обвенчальна  сваха  сабля вострая   калена стрела венчала среди битвы роковой  вижу смерть  приходит  чёрный ворон  весь  твой  как то летом  рассвете заглянул  соседний сад   смуглянка молдаванка собирает виноград   краснею   бледнею  захотелось  сказать   станем  рекою зорьки летние встречать   припев  раскудрявый клён зелёный  лист резной   влюблённый  смущённый пред тобой  клён зелёный   клён кудрявый   раскудрявый  резной  раскудрявый клён зелёный  лист резной   влюблённый  смущённый пред тобой  клён зелёный   клён кудрявый   раскудрявый  резной   

In [92]:
gram_tree = ngrams(song_data_wthot_punkt.split(), 2) # биграмма

for gram in gram_tree:
    print(gram)

('чёрный', 'ворон')
('ворон', 'чёрный')
('чёрный', 'ворон')
('ворон', 'вьёшься')
('вьёшься', 'мной')
('мной', 'добычи')
('добычи', 'дождёшься')
('дождёшься', 'чёрный')
('чёрный', 'ворон')
('ворон', 'твой')
('твой', 'когти')
('когти', 'распускаешь')
('распускаешь', 'моею')
('моею', 'головой')
('головой', 'добычу')
('добычу', 'чаешь')
('чаешь', 'чёрный')
('чёрный', 'ворон')
('ворон', 'твой')
('твой', 'завяжу')
('завяжу', 'смертельну')
('смертельну', 'рану')
('рану', 'подарённым')
('подарённым', 'платком')
('платком', 'тобой')
('тобой', 'стану')
('стану', 'говорить')
('говорить', 'одном')
('одном', 'полети')
('полети', 'мою')
('мою', 'сторонку')
('сторонку', 'скажи')
('скажи', 'маменьке')
('маменьке', 'моей')
('моей', 'скажи')
('скажи', 'моей')
('моей', 'любезной')
('любезной', 'родину')
('родину', 'пал')
('пал', 'отнеси')
('отнеси', 'платок')
('платок', 'кровавый')
('кровавый', 'милой')
('милой', 'любушке')
('любушке', 'моей')
('моей', 'скажи')
('скажи', 'свободна')
('свободна', 'женился

# pymorphy3

In [94]:
# POS Посмотрим варианты морфологического разбора слова:
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

In [95]:
# Посмотрим варианты морфологического разбора слова:
morph.parse('завирусилась')

[Parse(word='завирусилась', tag=OpencorporaTag('VERB,perf,intr femn,sing,past,indc'), normal_form='завируситься', score=1.0, methods_stack=((FakeDictionary(), 'вирусилась', 652, 2), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'илась'), (KnownPrefixAnalyzer(known_prefixes=<...>, min_remainder_length=3, score_multiplier=0.75), 'за')))]

In [96]:
# Получим наиболее вероятный тег и часть речи:
tag = morph.parse('стали')[0].tag
print(tag)
print(tag.POS)

VERB,perf,intr plur,past,indc
VERB


In [97]:
# Получим наиболее вероятный тег и часть речи:
tag = morph.parse('корешок')[0].tag
print(tag)
print(tag.POS)

NOUN,inan,masc sing,accs
NOUN


In [101]:
morph.parse('корешок')

[Parse(word='корешок', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='корешок', score=0.8, methods_stack=((DictionaryAnalyzer(), 'корешок', 141, 3),)),
 Parse(word='корешок', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='корешок', score=0.2, methods_stack=((DictionaryAnalyzer(), 'корешок', 141, 0),))]

In [100]:
morph.parse('корешок')[1].normal_form

'корешок'

In [98]:
# Приведение к нормальной форме:
morph.parse('думающему')[0].normal_form

'думать'

## Токенизация

In [165]:
sentences = sent_tokenize(song_data) # Разделение на предложения
sentences

['Чёрный ворон, чёрный ворон, Что ты вьёшься надо мной?',
 'Ты добычи не дождёшься, Чёрный ворон, я не твой!',
 'Что ты когти распускаешь Над моею головой?',
 'Иль добычу себе чаешь, Чёрный ворон, я не твой!',
 'Завяжу смертельну рану Подарённым мне платком, А потом с тобой я стану Говорить всё об одном.',
 'Полети в мою сторонку, Скажи маменьке моей, Ты скажи моей любезной, Что за Родину я пал.',
 'Отнеси платок кровавый Милой любушке моей.',
 'Ты скажи – она свободна, Я женился на другой.',
 'Взял невесту тиху-скромну В чистом поле под кустом, Обвенчальна была сваха – Сабля вострая моя.',
 'Калена стрела венчала Среди битвы роковой.',
 'Вижу смерть моя приходит – Чёрный ворон, весь я твой!',
 'Как-то летом на рассвете Заглянул в соседний сад, Там смуглянка-молдаванка Собирает виноград.',
 'Я краснею, я бледнею, Захотелось вдруг сказать: «Станем над рекою Зорьки летние встречать!»  Припев:  Раскудрявый клён зелёный, лист резной, Я влюблённый и смущённый пред тобой.',
 'Клён зелёный, д

In [167]:
# и разобьём на слова
words = word_tokenize(song_data_wthot_punkt) # Шаг 2. Разделение на слова
words

['чёрный',
 'ворон',
 'чёрный',
 'ворон',
 'вьёшься',
 'мной',
 'добычи',
 'дождёшься',
 'чёрный',
 'ворон',
 'твой',
 'когти',
 'распускаешь',
 'моею',
 'головой',
 'добычу',
 'чаешь',
 'чёрный',
 'ворон',
 'твой',
 'завяжу',
 'смертельну',
 'рану',
 'подарённым',
 'платком',
 'тобой',
 'стану',
 'говорить',
 'одном',
 'полети',
 'мою',
 'сторонку',
 'скажи',
 'маменьке',
 'моей',
 'скажи',
 'моей',
 'любезной',
 'родину',
 'пал',
 'отнеси',
 'платок',
 'кровавый',
 'милой',
 'любушке',
 'моей',
 'скажи',
 'свободна',
 'женился',
 'взял',
 'невесту',
 'тиху',
 'скромну',
 'чистом',
 'поле',
 'кустом',
 'обвенчальна',
 'сваха',
 'сабля',
 'вострая',
 'калена',
 'стрела',
 'венчала',
 'среди',
 'битвы',
 'роковой',
 'вижу',
 'смерть',
 'приходит',
 'чёрный',
 'ворон',
 'весь',
 'твой',
 'как',
 'то',
 'летом',
 'рассвете',
 'заглянул',
 'соседний',
 'сад',
 'смуглянка',
 'молдаванка',
 'собирает',
 'виноград',
 'краснею',
 'бледнею',
 'захотелось',
 'сказать',
 'станем',
 'рекою',
 'зор

In [168]:
full_words = word_tokenize(song_data_wthot_punkt) # сколько всего слов
len(full_words)

281

## Лемматизация

In [189]:
#@title Определить функцию лемматизации
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = []

    for word in words:
        # Удаляем знаки препинания
        clean_word = ''.join(e for e in word if e.isalnum())

        if clean_word:
            # Получаем лемму слова
            lemma = morph.parse(clean_word)[0].normal_form
            lemmatized_words.append(lemma)

    return ' '.join(lemmatized_words)

Эта функция делает следующее:

* Разбивает входной текст на отдельные слова.

* Для каждого слова удаляет знаки препинания.

* Использует pymorphy2 для получения нормальной формы (леммы) слова.

* Объединяет все лемматизированные слова обратно в строку.

Теперь мы можем использовать эту функцию для лемматизации текста:

In [190]:
lemmatized_text = lemmatize_text(song_data_wthot_punkt)

In [191]:
lemmatized_text

'чёрный ворон чёрный ворон виться я добыча дождаться чёрный ворон твой коготь распускать мой голова добыча чаять чёрный ворон твой завязать смертельнуть рана подарить платок ты стать говорить один полететь мой сторонка сказать маменька мой сказать мой любезный родина пасть отнести платок кровавый милый любушка мой сказать свободный жениться взять невеста тих скромять чистый поле куст обвенчальный сваха сабля вострый кален стрела венчать среди битва роковой видеть смерть приходить чёрный ворон весь твой как то лето рассвет заглянуть соседний сад смуглянка молдаванка собирать виноград краснеть бледнеть захотеться сказать стать река зорька летний встречать припев раскудрявый клён зелёный лист резной влюбить смутить пред ты клён зелёный клён кудрявый раскудрявый резной раскудрявый клён зелёный лист резной влюбить смутить пред ты клён зелёный клён кудрявый раскудрявый резной смуглянка молдаванка отвечать парень лад партизанский молдаванский собираться отряд нынче рано партизан дом покинуть 

## POS тэггинг

Создадим функцию, которая будет принимать текст и возвращать список кортежей, где каждый кортеж содержит слово и его POS-тег:



In [192]:
def pos_tagging(text):
    words = text.split()
    tagged_words = []

    for word in words:
        # Удаляем знаки препинания
        clean_word = ''.join(e for e in word if e.isalnum())

        if clean_word:
            # Получаем теги слова
            parsed_word = morph.parse(clean_word)[0]

            # Извлекаем основной тег части речи
            pos_tag = parsed_word.tag.POS

            tagged_words.append((word, pos_tag))

    return tagged_words


In [193]:
tagged_text = pos_tagging(lemmatized_text)

pymorphy3 использует следующую систему тегов частей речи:

ADJF: прилагательное (полное)

ADJS: прилагательное (краткое)

COMP: компаратив

CONJ: союз

GRND: деепричастие

INFN: инфинитив

INTG: междометие

NOUN: существительное

NUMR: числительное

PRED: предикатив

PREP: предлог

PRCL: частица

PRTF: причастие (полное)

PRTS: причастие (краткое)

VERB: глагол

In [194]:
tagged_text

[('чёрный', 'ADJF'),
 ('ворон', 'NOUN'),
 ('чёрный', 'ADJF'),
 ('ворон', 'NOUN'),
 ('виться', 'INFN'),
 ('я', 'NPRO'),
 ('добыча', 'NOUN'),
 ('дождаться', 'INFN'),
 ('чёрный', 'ADJF'),
 ('ворон', 'NOUN'),
 ('твой', 'ADJF'),
 ('коготь', 'NOUN'),
 ('распускать', 'INFN'),
 ('мой', 'ADJF'),
 ('голова', 'NOUN'),
 ('добыча', 'NOUN'),
 ('чаять', 'INFN'),
 ('чёрный', 'ADJF'),
 ('ворон', 'NOUN'),
 ('твой', 'ADJF'),
 ('завязать', 'INFN'),
 ('смертельнуть', 'INFN'),
 ('рана', 'NOUN'),
 ('подарить', 'INFN'),
 ('платок', 'NOUN'),
 ('ты', 'NPRO'),
 ('стать', 'INFN'),
 ('говорить', 'INFN'),
 ('один', 'ADJF'),
 ('полететь', 'INFN'),
 ('мой', 'ADJF'),
 ('сторонка', 'NOUN'),
 ('сказать', 'INFN'),
 ('маменька', 'NOUN'),
 ('мой', 'ADJF'),
 ('сказать', 'INFN'),
 ('мой', 'ADJF'),
 ('любезный', 'ADJF'),
 ('родина', 'NOUN'),
 ('пасть', 'INFN'),
 ('отнести', 'INFN'),
 ('платок', 'NOUN'),
 ('кровавый', 'ADJF'),
 ('милый', 'ADJF'),
 ('любушка', 'NOUN'),
 ('мой', 'ADJF'),
 ('сказать', 'INFN'),
 ('свободный', 'ADJ

## Анализ n-грамм

In [195]:
def analyze_ngrams(text, n):
    # Токенизация текста
    tokens = nltk.word_tokenize(text)

    # Создание n-грамм
    ngrams = nltk.ngrams(tokens, n)

    # Лемматизация и получение частей речи
    lemmatized_ngrams = []
    for gram in ngrams:
        lemmatized_gram = []
        for word in gram:
            clean_word = ''.join(e for e in word if e.isalnum())
            if clean_word:
                parsed_word = morph.parse(clean_word)[0]
                lemma = parsed_word.normal_form
                pos_tag = parsed_word.tag.POS
                lemmatized_gram.append((lemma, pos_tag))
        lemmatized_ngrams.append(lemmatized_gram)

    # Подсчет частоты n-грамм
    ngram_freq = Counter(tuple(ngram) for ngram in lemmatized_ngrams)

    return ngram_freq


In [198]:
n = 2 # количество n-грамм

result = analyze_ngrams(lemmatized_text, n) # n-граммы и сколько раз они используются
result

Counter({(('чёрный', 'ADJF'), ('ворон', 'NOUN')): 5,
         (('ворон', 'NOUN'), ('чёрный', 'ADJF')): 1,
         (('ворон', 'NOUN'), ('виться', 'INFN')): 1,
         (('виться', 'INFN'), ('я', 'NPRO')): 1,
         (('я', 'NPRO'), ('добыча', 'NOUN')): 1,
         (('добыча', 'NOUN'), ('дождаться', 'INFN')): 1,
         (('дождаться', 'INFN'), ('чёрный', 'ADJF')): 1,
         (('ворон', 'NOUN'), ('твой', 'ADJF')): 2,
         (('твой', 'ADJF'), ('коготь', 'NOUN')): 1,
         (('коготь', 'NOUN'), ('распускать', 'INFN')): 1,
         (('распускать', 'INFN'), ('мой', 'ADJF')): 1,
         (('мой', 'ADJF'), ('голова', 'NOUN')): 1,
         (('голова', 'NOUN'), ('добыча', 'NOUN')): 1,
         (('добыча', 'NOUN'), ('чаять', 'INFN')): 1,
         (('чаять', 'INFN'), ('чёрный', 'ADJF')): 1,
         (('твой', 'ADJF'), ('завязать', 'INFN')): 1,
         (('завязать', 'INFN'), ('смертельнуть', 'INFN')): 1,
         (('смертельнуть', 'INFN'), ('рана', 'NOUN')): 1,
         (('рана', 'NOUN'), 

# Natasha

Библиотеки. Natasha. Решает базовые задачи обработки естественного русского языка: сегментация на токены и предложения, морфологический и синтаксический анализ, лемматизация, извлечение, нормализация именованных сущностей.

In [102]:
emb = NewsEmbedding() # определение эмбеддинга
syntax_parser = NewsSyntaxParser(emb) # определение смысла
morph_tagger = NewsMorphTagger(emb) # тэги слов
morph_vocab = MorphVocab() # объект для лемматизации
segmenter = Segmenter() # сегментация
ner_tagger = NewsNERTagger(emb) # задаче распознавания именованных сущностей (NER)

In [103]:
test = "Мне нравится программирование на Python"
doc = Doc(test) # создание объекта документа

In [104]:
doc

Doc(text='Мне нравится программирование на Python')

In [105]:
doc_tree = Doc(song_data_wthot_punkt)
doc_tree

Doc(text='чёрный ворон  чёрный ворон    вьёшься  мной   доб...)

In [107]:
doc_tree_full = Doc(song_data)
doc_tree_full

Doc(text='Чёрный ворон, чёрный ворон, Что ты вьёшься надо м...)

In [108]:
doc.segment(segmenter) # загрузка segmenter
doc_tree.segment(segmenter) # загрузка segmenter
doc_tree_full.segment(segmenter) # загрузка segmenter

In [112]:
print(doc.tokens) # не работает без сегментации

[DocToken(stop=3, text='Мне'), DocToken(start=4, stop=12, text='нравится'), DocToken(start=13, stop=29, text='программирование'), DocToken(start=30, stop=32, text='на'), DocToken(start=33, stop=39, text='Python')]


In [110]:
print(doc_tree.tokens[:5]) # не работает без сегментации

[DocToken(stop=6, text='чёрный'), DocToken(start=7, stop=12, text='ворон'), DocToken(start=14, stop=20, text='чёрный'), DocToken(start=21, stop=26, text='ворон'), DocToken(start=30, stop=37, text='вьёшься')]


In [111]:
print(doc_tree_full.tokens[:5]) # не работает без сегментации

[DocToken(stop=6, text='Чёрный'), DocToken(start=7, stop=12, text='ворон'), DocToken(start=12, stop=13, text=','), DocToken(start=14, stop=20, text='чёрный'), DocToken(start=21, stop=26, text='ворон')]


In [113]:
print(doc.sents[:5])  # не работает без сегментации

[DocSent(stop=39, text='Мне нравится программирование на Python', tokens=[...])]


In [114]:
print(doc_tree.sents[:5]) # не работает без сегментации

[DocSent(stop=2296, text='чёрный ворон  чёрный ворон    вьёшься  мной   доб..., tokens=[...])]


In [115]:
print(doc_tree_full.sents[:5]) # не работает без сегментации

[DocSent(stop=53, text='Чёрный ворон, чёрный ворон, Что ты вьёшься надо м..., tokens=[...]), DocSent(start=54, stop=102, text='Ты добычи не дождёшься, Чёрный ворон, я не твой!', tokens=[...]), DocSent(start=104, stop=146, text='Что ты когти распускаешь Над моею головой?', tokens=[...]), DocSent(start=147, stop=194, text='Иль добычу себе чаешь, Чёрный ворон, я не твой!', tokens=[...]), DocSent(start=196, stop=289, text='Завяжу смертельну рану Подарённым мне платком, А ..., tokens=[...])]


In [116]:
tree_sent_doc = Doc(song_data)
tree_sent_doc.segment(segmenter)

In [117]:
tree_sent_doc.sents[:5]  # не работает без сегментации

[DocSent(stop=53, text='Чёрный ворон, чёрный ворон, Что ты вьёшься надо м..., tokens=[...]),
 DocSent(start=54, stop=102, text='Ты добычи не дождёшься, Чёрный ворон, я не твой!', tokens=[...]),
 DocSent(start=104, stop=146, text='Что ты когти распускаешь Над моею головой?', tokens=[...]),
 DocSent(start=147, stop=194, text='Иль добычу себе чаешь, Чёрный ворон, я не твой!', tokens=[...]),
 DocSent(start=196, stop=289, text='Завяжу смертельну рану Подарённым мне платком, А ..., tokens=[...])]

In [118]:
print(len(tree_sent_doc.sents[0].text))  # не работает без сегментации

53


In [119]:
test = tree_sent_doc.sents[:5][0]

In [121]:
test

DocSent(stop=53, text='Чёрный ворон, чёрный ворон, Что ты вьёшься надо м..., tokens=[...])

In [123]:
#morph_tagger = NewsMorphTagger(emb) # тэги слов
doc.tag_morph(morph_tagger)

In [125]:
doc.tokens[:5]

[DocToken(stop=3, text='Мне', pos='PRON', feats=<Dat,Sing,1>),
 DocToken(start=4, stop=12, text='нравится', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Mid>),
 DocToken(start=13, stop=29, text='программирование', pos='NOUN', feats=<Inan,Nom,Neut,Sing>),
 DocToken(start=30, stop=32, text='на', pos='ADP'),
 DocToken(start=33, stop=39, text='Python', pos='X', feats=<Yes>)]

In [126]:
tree_sent_doc.tag_morph(morph_tagger)

In [128]:
doc_tree.tag_morph(morph_tagger)

In [127]:
tree_sent_doc.tokens[:10]

[DocToken(stop=6, text='Чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>),
 DocToken(start=7, stop=12, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=12, stop=13, text=',', pos='PUNCT'),
 DocToken(start=14, stop=20, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>),
 DocToken(start=21, stop=26, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=26, stop=27, text=',', pos='PUNCT'),
 DocToken(start=28, stop=31, text='Что', pos='PRON', feats=<Inan,Acc,Neut,Sing>),
 DocToken(start=32, stop=34, text='ты', pos='PRON', feats=<Nom,Sing,2>),
 DocToken(start=35, stop=42, text='вьёшься', pos='VERB', feats=<Imp,Ind,Sing,2,Pres,Fin,Act>),
 DocToken(start=43, stop=47, text='надо', pos='ADV', feats=<Pos>)]

In [129]:
doc_tree.tokens[:10]

[DocToken(stop=6, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>),
 DocToken(start=7, stop=12, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=14, stop=20, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>),
 DocToken(start=21, stop=26, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=30, stop=37, text='вьёшься', pos='VERB', feats=<Perf,Ind,Sing,3,Fut,Fin,Act>),
 DocToken(start=39, stop=43, text='мной', pos='PRON', feats=<Ins,Sing,1>),
 DocToken(start=46, stop=52, text='добычи', pos='NOUN', feats=<Inan,Gen,Fem,Sing>),
 DocToken(start=54, stop=63, text='дождёшься', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Act>),
 DocToken(start=65, stop=71, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>),
 DocToken(start=72, stop=77, text='ворон', pos='NOUN', feats=<Inan,Nom,Masc,Sing>)]

In [131]:
doc_tree.tokens

[DocToken(stop=6, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='черный'),
 DocToken(start=7, stop=12, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>, lemma='ворон'),
 DocToken(start=14, stop=20, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='черный'),
 DocToken(start=21, stop=26, text='ворон', pos='NOUN', feats=<Anim,Nom,Masc,Sing>, lemma='ворон'),
 DocToken(start=30, stop=37, text='вьёшься', pos='VERB', feats=<Perf,Ind,Sing,3,Fut,Fin,Act>, lemma='виться'),
 DocToken(start=39, stop=43, text='мной', pos='PRON', feats=<Ins,Sing,1>, lemma='я'),
 DocToken(start=46, stop=52, text='добычи', pos='NOUN', feats=<Inan,Gen,Fem,Sing>, lemma='добыча'),
 DocToken(start=54, stop=63, text='дождёшься', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Act>, lemma='дождаться'),
 DocToken(start=65, stop=71, text='чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='черный'),
 DocToken(start=72, stop=77, text='ворон', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='ворон'),
 DocT

In [132]:
for token in doc_tree.tokens:
  token.lemmatize(morph_vocab) # morph_vocab = MorphVocab() # объект для лемматизации

In [133]:
print(doc.tokens[:5])

[DocToken(stop=3, text='Мне', pos='PRON', feats=<Dat,Sing,1>), DocToken(start=4, stop=12, text='нравится', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Mid>), DocToken(start=13, stop=29, text='программирование', pos='NOUN', feats=<Inan,Nom,Neut,Sing>), DocToken(start=30, stop=32, text='на', pos='ADP'), DocToken(start=33, stop=39, text='Python', pos='X', feats=<Yes>)]


In [134]:
for token in doc.tokens:
  token.lemmatize(morph_vocab)

In [135]:
{_.text: _.lemma for _ in doc.tokens}

{'Мне': 'я',
 'нравится': 'нравиться',
 'программирование': 'программирование',
 'на': 'на',
 'Python': 'python'}

In [136]:
{_.text: _.lemma for _ in doc_tree.tokens}

{'чёрный': 'черный',
 'ворон': 'ворон',
 'вьёшься': 'виться',
 'мной': 'я',
 'добычи': 'добыча',
 'дождёшься': 'дождаться',
 'твой': 'твой',
 'когти': 'коготь',
 'распускаешь': 'распускать',
 'моею': 'мой',
 'головой': 'голова',
 'добычу': 'добыча',
 'чаешь': 'чаять',
 'завяжу': 'завязать',
 'смертельну': 'смертельна',
 'рану': 'рана',
 'подарённым': 'подарить',
 'платком': 'платок',
 'тобой': 'ты',
 'стану': 'стать',
 'говорить': 'говорить',
 'одном': 'один',
 'полети': 'полететь',
 'мою': 'мой',
 'сторонку': 'сторонка',
 'скажи': 'сказать',
 'маменьке': 'маменька',
 'моей': 'мой',
 'любезной': 'любезный',
 'родину': 'родина',
 'пал': 'пасть',
 'отнеси': 'отнести',
 'платок': 'платок',
 'кровавый': 'кровавый',
 'милой': 'милый',
 'любушке': 'любушка',
 'свободна': 'свободный',
 'женился': 'жениться',
 'взял': 'взять',
 'невесту': 'невеста',
 'тиху': 'тих',
 'скромну': 'скромну',
 'чистом': 'чистый',
 'поле': 'поле',
 'кустом': 'куст',
 'обвенчальна': 'обвенчальный',
 'сваха': 'сваха',

In [138]:
doc.parse_syntax(syntax_parser) # синтаксический анализатор

In [139]:
doc.tokens[:5]

[DocToken(stop=3, text='Мне', id='1_1', head_id='1_2', rel='iobj', pos='PRON', feats=<Dat,Sing,1>, lemma='я'),
 DocToken(start=4, stop=12, text='нравится', id='1_2', head_id='1_0', rel='root', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Mid>, lemma='нравиться'),
 DocToken(start=13, stop=29, text='программирование', id='1_3', head_id='1_2', rel='nsubj', pos='NOUN', feats=<Inan,Nom,Neut,Sing>, lemma='программирование'),
 DocToken(start=30, stop=32, text='на', id='1_4', head_id='1_5', rel='case', pos='ADP', lemma='на'),
 DocToken(start=33, stop=39, text='Python', id='1_5', head_id='1_3', rel='nmod', pos='X', feats=<Yes>, lemma='python')]

In [140]:
doc.sents[0].syntax.print()

    ┌► Мне              iobj
  ┌─└─ нравится         
┌─└──► программирование nsubj
│   ┌► на               case
└──►└─ Python           nmod


In [141]:
doc_tree.parse_syntax(syntax_parser) # синтаксический анализатор

In [143]:
doc_tree.tokens[:15]

[DocToken(stop=6, text='чёрный', id='1_1', head_id='1_10', rel='amod', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='черный'),
 DocToken(start=7, stop=12, text='ворон', id='1_2', head_id='1_127', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Sing>, lemma='ворон'),
 DocToken(start=14, stop=20, text='чёрный', id='1_3', head_id='1_4', rel='amod', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='черный'),
 DocToken(start=21, stop=26, text='ворон', id='1_4', head_id='1_190', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Sing>, lemma='ворон'),
 DocToken(start=30, stop=37, text='вьёшься', id='1_5', head_id='1_131', rel='acl', pos='VERB', feats=<Perf,Ind,Sing,3,Fut,Fin,Act>, lemma='виться'),
 DocToken(start=39, stop=43, text='мной', id='1_6', head_id='1_5', rel='obl', pos='PRON', feats=<Ins,Sing,1>, lemma='я'),
 DocToken(start=46, stop=52, text='добычи', id='1_7', head_id='1_190', rel='obj', pos='NOUN', feats=<Inan,Gen,Fem,Sing>, lemma='добыча'),
 DocToken(start=54, stop=63, text='дождёшься', id='1

In [144]:
doc_tree.sents[0].syntax.print()

                                                                                                                                                                                                                                                                                        ┌──► чёрный       amod
                                                                                                                          ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────►│    ворон        nsubj
                                                                                                                          │                                                                                                                                                             │ ┌► чёрный       amod
                                  ┌───────────────────────────────────────────────────────

In [147]:
doc_tree_full.parse_syntax(syntax_parser) # синтаксический анализатор

In [148]:
doc_tree_full.tokens[:15]

[DocToken(stop=6, text='Чёрный', id='1_1', head_id='1_2', rel='amod'),
 DocToken(start=7, stop=12, text='ворон', id='1_2', head_id='1_10', rel='nsubj'),
 DocToken(start=12, stop=13, text=',', id='1_3', head_id='1_5', rel='punct'),
 DocToken(start=14, stop=20, text='чёрный', id='1_4', head_id='1_5', rel='amod'),
 DocToken(start=21, stop=26, text='ворон', id='1_5', head_id='1_2', rel='conj'),
 DocToken(start=26, stop=27, text=',', id='1_6', head_id='1_9', rel='punct'),
 DocToken(start=28, stop=31, text='Что', id='1_7', head_id='1_9', rel='obj'),
 DocToken(start=32, stop=34, text='ты', id='1_8', head_id='1_9', rel='nsubj'),
 DocToken(start=35, stop=42, text='вьёшься', id='1_9', head_id='1_0', rel='root'),
 DocToken(start=43, stop=47, text='надо', id='1_10', head_id='1_0', rel='root'),
 DocToken(start=48, stop=52, text='мной', id='1_11', head_id='1_10', rel='csubj'),
 DocToken(start=52, stop=53, text='?', id='1_12', head_id='1_10', rel='punct'),
 DocToken(start=54, stop=56, text='Ты', id='

In [149]:
doc_tree_full.sents[0].syntax.print()

      ┌► Чёрный  amod
┌►┌───└─ ворон   nsubj
│ │ ┌──► ,       punct
│ │ │ ┌► чёрный  amod
│ └►└─└─ ворон   conj
│ ┌────► ,       punct
│ │ ┌──► Что     obj
│ │ │ ┌► ты      nsubj
│ └─└─└─ вьёшься 
└───┌─┌─ надо    
    │ └► мной    csubj
    └──► ?       punct


In [150]:
doc.tag_ner(ner_tagger) # standart named entities: names, locations, organizations

In [151]:
print(doc.spans[:5])

[]


In [152]:
doc.ner.print()

Мне нравится программирование на Python


In [153]:
for span in doc.spans:
  span.normalize(morph_vocab)

In [154]:
print(doc.spans[:5])

[]


In [155]:
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{}

In [156]:
tree_sent_doc.tag_ner(ner_tagger) # standart named entities: names, locations, organizations

In [157]:
tree_sent_doc.ner.print()

Чёрный ворон, чёрный ворон, Что ты вьёшься надо мной? Ты добычи не 
дождёшься, Чёрный ворон, я не твой!  Что ты когти распускаешь Над моею
           PER───                                             PER     
 головой? Иль добычу себе чаешь, Чёрный ворон, я не твой!  Завяжу 
                                 PER───                           
смертельну рану Подарённым мне платком, А потом с тобой я стану 
                PER───────                                      
Говорить всё об одном.  Полети в мою сторонку, Скажи маменьке моей, Ты
                                               PER──                  
 скажи моей любезной, Что за Родину я пал.  Отнеси платок кровавый 
Милой любушке моей. Ты скажи – она свободна, Я женился на другой.  
PER──                                                              
Взял невесту тиху-скромну В чистом поле под кустом, Обвенчальна была 
                                                    PER────────      
сваха – Сабля вострая моя.  Калена стрел

In [158]:
# Более полный пример разбора
tt = 'Появление ООН было обусловлено целым рядом объективных факторов'


emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(tt)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for token in doc.tokens:
    print(token.pos, token.text)

NOUN Появление
PROPN ООН
AUX было
VERB обусловлено
ADJ целым
NOUN рядом
ADJ объективных
NOUN факторов


In [159]:
tree_sent_doc.tokens[0]

DocToken(stop=6, text='Чёрный', pos='ADJ', feats=<Nom,Pos,Masc,Sing>)

In [160]:
tree_sent_doc.tag_morph(morph_tagger)

for token in tree_sent_doc.tokens:
    print(token.pos, token.text)

ADJ Чёрный
NOUN ворон
PUNCT ,
ADJ чёрный
NOUN ворон
PUNCT ,
PRON Что
PRON ты
VERB вьёшься
ADV надо
PRON мной
PUNCT ?
PRON Ты
NOUN добычи
PART не
VERB дождёшься
PUNCT ,
ADJ Чёрный
NOUN ворон
PUNCT ,
PRON я
PART не
DET твой
PUNCT !
PRON Что
PRON ты
NOUN когти
VERB распускаешь
ADP Над
ADJ моею
NOUN головой
PUNCT ?
PROPN Иль
NOUN добычу
PRON себе
NOUN чаешь
PUNCT ,
ADJ Чёрный
NOUN ворон
PUNCT ,
PRON я
PART не
DET твой
PUNCT !
PROPN Завяжу
ADJ смертельну
NOUN рану
PROPN Подарённым
PRON мне
NOUN платком
PUNCT ,
PROPN А
ADV потом
ADP с
PRON тобой
PRON я
VERB стану
VERB Говорить
PRON всё
ADP об
NUM одном
PUNCT .
PROPN Полети
ADP в
DET мою
NOUN сторонку
PUNCT ,
PROPN Скажи
VERB маменьке
DET моей
PUNCT ,
PRON Ты
NOUN скажи
DET моей
NOUN любезной
PUNCT ,
PRON Что
ADP за
PROPN Родину
PRON я
VERB пал
PUNCT .
ADJ Отнеси
ADJ платок
ADJ кровавый
PROPN Милой
ADJ любушке
DET моей
PUNCT .
PRON Ты
VERB скажи
PUNCT –
PRON она
ADJ свободна
PUNCT ,
PRON Я
VERB женился
ADP на
ADJ другой
PUNCT .
VERB Взял
NOUN